In [1]:
try:
    import os
    import sys
    import elasticsearch
    from elasticsearch import Elasticsearch
    import pandas as pd 
    import numpy as np
    import json
    from ast import literal_eval
    from tqdm import tqdm 
    import datetime
    from elasticsearch import helpers
    from pathlib import Path
    
except Exception as e:
    print(f"error: {e}")

In [2]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch
es.ping()

C:\Users\HamidRezaAttar\anaconda3\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
GET http://localhost:9200/ [status:200 request:0.003s]
HEAD http://localhost:9200/ [status:200 request:0.001s]


True

In [3]:
# Importing test dataset
ROOT_DIR = Path('.').resolve().parent
DATA_DIR = ROOT_DIR / "data"

dataset = pd.read_csv(str(DATA_DIR / "test.csv.gz"), compression="gzip")
dataset.drop("id", axis=1, inplace=True)
print(f"shape of dataset: {dataset.shape}")
dataset.head()

shape of dataset: (11490, 2)


article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   

                                          highlights  
0  Experts question if  packed out planes are put...  
1  Drunk teenage boy climbed into lion enclosure ...  
2  Nottingham Forest are close to extending Dougi...  
3  Fiorentina goalkeeper Neto has been linked wit...  
4  Tell-all interview with the reality TV star, 6...

In [4]:
# Define settings & mappings of Elasticsearch index
Settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "title":{
                "type":"text"
            },
            "abstract":{
                "type":"text"
            }
        }
    }
}

In [5]:
def json_formatter(dataset, index_name, index_type='_doc'):
    """
    This function is used to create JSON formatted dictionaries for Elasticsearch.

    Args:
      dataset: The dataset you want to apply this function.
      index_name: Name of the index in elasticsearch
      index_type: Type of the index in elasticsearch.
      Note: It is suggested to keep index_type as '_doc', since it is deprecated from version 6.
      Note: This function formats all columns of your dataset, if you want to apply this to special columns only,
      you can delete the second for loop and add your custom fields.
    """
    try:
        List = []
        columns = dataset.columns
        for idx, row in dataset.iterrows():
            dic = {}
            dic['_index'] = index_name
            dic['_type'] = index_type
            source = {}
            for i in dataset.columns:
                source[i] = row[i]
            dic['_source'] = source
            List.append(dic)
        return List
    
    except Exception as e:
        print("There is a problem: {}".format(e))

In [6]:
MY_INDEX = es.indices.create(index="news_index", ignore=[400,404], body=Settings)
MY_INDEX

C:\Users\HamidRezaAttar\anaconda3\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
PUT http://localhost:9200/news_index [status:200 request:0.204s]


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'news_index'}

In [7]:
json_Formatted_dataset = json_formatter(dataset=dataset, index_name='news_index', index_type='_doc')
json_Formatted_dataset[0]

{'_index': 'news_index',
 '_type': '_doc',
 '_source': {'article': "Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could 

In [8]:
#For importing Data to elasticsearch we use elasticsearch's bulk API from elasticsearch.helpers
try:
    res = helpers.bulk(es, json_Formatted_dataset)
    print("successfully imported to elasticsearch.")
except Exception as e:
    print(f"error: {e}")

C:\Users\HamidRezaAttar\anaconda3\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)
POST http://localhost:9200/_bulk [status:200 request:0.241s]
POST http://localhost:9200/_bulk [status:200 request:0.234s]
POST http://localhost:9200/_bulk [status:200 request:0.170s]
POST http://localhost:9200/_bulk [status:200 request:0.144s]
POST http://localhost:9200/_bulk [status:200 request:0.157s]
POST http://localhost:9200/_bulk [status:200 request:0.152s]
POST http://localhost:9200/_bulk [status:200 request:0.186s]
POST http://localhost:9200/_bulk [status:200 request:0.203s]
POST http://localhost:9200/_bulk [status:200 request:0.170s]
POST http://localhost:9200/_bulk [status:200 request:0.146s]
POST http://localhost:9200/_bulk [status:200 request:0.200s]
POST http://localhost:9200/_bulk [status:200 request:0.159s]
POST http://localhost:9200/_bulk [s

successfully imported to elasticsearch.


In [9]:
# Get 10 sample of data
query = es.search(
    index="news_index",
    body={
      "size":10,
      "query": {
        "match_all":{}
      }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

POST http://localhost:9200/news_index/_search [status:200 request:0.008s]


_index _type                   _id  _score  \
0  news_index  _doc  YXDWq38B8oXRx87H6D6O     1.0   
1  news_index  _doc  YnDWq38B8oXRx87H6D6O     1.0   
2  news_index  _doc  Y3DWq38B8oXRx87H6D6O     1.0   
3  news_index  _doc  ZHDWq38B8oXRx87H6D6O     1.0   
4  news_index  _doc  ZXDWq38B8oXRx87H6D6O     1.0   
5  news_index  _doc  ZnDWq38B8oXRx87H6D6O     1.0   
6  news_index  _doc  Z3DWq38B8oXRx87H6D6O     1.0   
7  news_index  _doc  aHDWq38B8oXRx87H6D6O     1.0   
8  news_index  _doc  aXDWq38B8oXRx87H6D6O     1.0   
9  news_index  _doc  anDWq38B8oXRx87H6D6O     1.0   

                                _ignored  \
0                      [article.keyword]   
1                      [article.keyword]   
2                      [article.keyword]   
3  [article.keyword, highlights.keyword]   
4  [article.keyword, highlights.keyword]   
5                      [article.keyword]   
6  [article.keyword, highlights.keyword]   
7                      [article.keyword]   
8  [article.keyword, highlights.keyword]   
9  [article.keyword, highlights.keyword]   

                                     _source.article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   
5  This is the moment that a crew of firefighters...   
6  The amount of time people spend listening to B...   
7  (CNN)So, you'd like a "Full House" reunion and...   
8  At 11:20pm, former world champion Ken Doherty ...   
9  A gang of six men have been jailed for a total...   

                                  _source.highlights  
0  Experts question if  packed out planes are put...  
1  Drunk teenage boy climbed into lion enclosure ...  
2  Nottingham Forest are close to extending Dougi...  
3  Fiorentina goalkeeper Neto has been linked wit...  
4  Tell-all interview with the reality TV star, 6...  
5  Giant pig fell into the swimming pool at his h...  
6  Figures show that while millions still tune in...  
7  Show will return with a one-hour special, foll...  
8  Reanne Evans faced Ken Doherty in World Champi...  
9  Gang have been jailed for a total of 31 years ...

In [10]:
# Complicated query
query = es.search(
    index="news_index",
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"match":{"article":"teenage boy"}}
                ],
                "should":[
                        {"match":{"highlights":"drunk"}}
                ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

C:\Users\HamidRezaAttar\anaconda3\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
POST http://localhost:9200/news_index/_search [status:200 request:0.018s]


_index _type                   _id     _score  \
0   news_index  _doc  YnDWq38B8oXRx87H6D6O  15.533424   
1   news_index  _doc  VnDWq38B8oXRx87H709j  11.526792   
2   news_index  _doc  W3DWq38B8oXRx87H-GUp  11.196183   
3   news_index  _doc  lHDWq38B8oXRx87H6UCd  10.722628   
4   news_index  _doc  CXDWq38B8oXRx87H8laH  10.383848   
5   news_index  _doc  e3DWq38B8oXRx87H-Gbq   9.660924   
6   news_index  _doc  EXDWq38B8oXRx87H7Egw   9.587729   
7   news_index  _doc  m3DWq38B8oXRx87H7Uqh   9.572909   
8   news_index  _doc  OXDWq38B8oXRx87H92Ji   9.094333   
9   news_index  _doc  THDWq38B8oXRx87H9V-x   9.034744   
10  news_index  _doc  Y3DWq38B8oXRx87H8FPe   9.025766   
11  news_index  _doc  K3DWq38B8oXRx87H92Nj   9.010388   
12  news_index  _doc  UnDWq38B8oXRx87H8FLe   8.876998   
13  news_index  _doc  SHDWq38B8oXRx87H-Win   8.718247   
14  news_index  _doc  zHDWq38B8oXRx87H7Uuh   8.679010   
15  news_index  _doc  Y3DWq38B8oXRx87H81hP   8.646388   
16  news_index  _doc  K3DWq38B8oXRx87H6UKd   8.574642   
17  news_index  _doc  ZnDWq38B8oXRx87H8VXK   8.497845   
18  news_index  _doc  hnDWq38B8oXRx87H7kx7   8.482735   
19  news_index  _doc  oHDWq38B8oXRx87H8FPe   8.343093   

                                 _ignored  \
0                       [article.keyword]   
1   [article.keyword, highlights.keyword]   
2                       [article.keyword]   
3   [article.keyword, highlights.keyword]   
4   [article.keyword, highlights.keyword]   
5   [article.keyword, highlights.keyword]   
6   [article.keyword, highlights.keyword]   
7   [article.keyword, highlights.keyword]   
8   [article.keyword, highlights.keyword]   
9   [article.keyword, highlights.keyword]   
10  [article.keyword, highlights.keyword]   
11  [article.keyword, highlights.keyword]   
12                      [article.keyword]   
13                      [article.keyword]   
14  [article.keyword, highlights.keyword]   
15  [article.keyword, highlights.keyword]   
16  [article.keyword, highlights.keyword]   
17                      [article.keyword]   
18                      [article.keyword]   
19  [article.keyword, highlights.keyword]   

                                      _source.article  \
0   A drunk teenage boy had to be rescued by secur...   
1   Teenage girls do better in single sex schools ...   
2   He looks barely teenage. But this child has am...   
3   A teenage boy who shot and killed a teacher an...   
4   A female school worker has been arrested on su...   
5   An alcohol-fuelled 14-year-old boy killed a fa...   
6   Teenagers across America got a very realistic ...   
7   The savages of Islamic State have stepped up t...   
8   A teenager who allegedly played naked Twister ...   
9   Forget about the 'theory of everything', an ev...   
10  The mother-of-five who was arrested for allege...   
11  A Saturday Night Live skit about a male studen...   
12  Jilted music teacher Yulia Simonova, 52, wante...   
13  A Florida woman has been charged with child ab...   
14  Kaitlyn Granado, 24, was arrested on Monday am...   
15  A gun-wielding mother has been arrested after ...   
16  Jurors have started deliberations in the case ...   
17  The naked bodies of a young couple were found ...   
18  In The Hunger Games, Katniss Everdeen is pulle...   
19  The axe wielding thug who attempted to rob a s...   

                                   _source.highlights  
0   Drunk teenage boy climbed into lion enclosure ...  
1   'Boy free' environment stops girls from being ...  
2   Child has amassed thousands of Twitter followe...  
3   Boy, 13, arrested after deadly attack at secon...  
4   Diane Blankenship, 45, was arrested at her hom...  
5   Kyle Major killed Paul Walker with a single pu...  
6   The Glasford school in Illinois hosted one of ...  
7   Teenage suicide bomber Abu Hafs al Badri is re...  
8   Rachel Lehnardt, 35, 'allowed her 16-year-old ...  
9   The legendary physicist was speaking at the Sy...  
10  Rachel Lehnardt, 35, 'allowed her 16-year-old 

In [11]:
# More complicated query
query = es.search( 
    index="news_index",
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"multi_match":{
                            "query":"The Hunger Games",
                            "fields":["article","highlights"]
                        }}
                ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

C:\Users\HamidRezaAttar\anaconda3\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
POST http://localhost:9200/news_index/_search [status:200 request:0.028s]


_index _type                   _id     _score  \
0   news_index  _doc  o3DWq38B8oXRx87H8FLe  14.259413   
1   news_index  _doc  hnDWq38B8oXRx87H7kx7  13.301999   
2   news_index  _doc  vnDWq38B8oXRx87H-GQp  11.486975   
3   news_index  _doc  SXDWq38B8oXRx87H8FPe  11.437349   
4   news_index  _doc  2XDWq38B8oXRx87H-GQp  11.259435   
5   news_index  _doc  pXDWq38B8oXRx87H6kOn  10.941918   
6   news_index  _doc  oXDWq38B8oXRx87H7Ejs  10.669691   
7   news_index  _doc  MHDWq38B8oXRx87H8FLe  10.634808   
8   news_index  _doc  93DWq38B8oXRx87H8lWH  10.627806   
9   news_index  _doc  6HDWq38B8oXRx87H8lWH  10.489234   
10  news_index  _doc  3XDWq38B8oXRx87H8FAq  10.430956   
11  news_index  _doc  yXDWq38B8oXRx87H8VXK   9.826724   
12  news_index  _doc  n3DWq38B8oXRx87H-Wen   8.775270   
13  news_index  _doc  DHDWq38B8oXRx87H8FLe   8.698811   
14  news_index  _doc  gXDWq38B8oXRx87H705j   8.503597   
15  news_index  _doc  AXDWq38B8oXRx87H8VTJ   8.387949   
16  news_index  _doc  AXDWq38B8oXRx87H-mqH   8.086366   
17  news_index  _doc  -nDWq38B8oXRx87H92Fi   7.953035   
18  news_index  _doc  WXDWq38B8oXRx87H6kOn   7.574566   
19  news_index  _doc  _nDWq38B8oXRx87H7kx7   7.413477   

                                 _ignored  \
0   [article.keyword, highlights.keyword]   
1                       [article.keyword]   
2   [article.keyword, highlights.keyword]   
3                       [article.keyword]   
4   [article.keyword, highlights.keyword]   
5                       [article.keyword]   
6   [article.keyword, highlights.keyword]   
7   [article.keyword, highlights.keyword]   
8   [article.keyword, highlights.keyword]   
9   [article.keyword, highlights.keyword]   
10  [article.keyword, highlights.keyword]   
11  [article.keyword, highlights.keyword]   
12  [article.keyword, highlights.keyword]   
13  [article.keyword, highlights.keyword]   
14  [article.keyword, highlights.keyword]   
15                      [article.keyword]   
16  [article.keyword, highlights.keyword]   
17                      [article.keyword]   
18                      [article.keyword]   
19  [article.keyword, highlights.keyword]   

                                      _source.article  \
0   If you were spellbound by the death-defying ad...   
1   In The Hunger Games, Katniss Everdeen is pulle...   
2   When complete strangers Niamh Geaney, 26, and ...   
3   (CNN)Universal's "Furious 7" continues to buil...   
4   Sky launched its Buy and Keep scheme for custo...   
5   (CNN)Universal's "Furious 7" is about to make ...   
6   Mourinho with his wife Matilde Faria at the 20...   
7   It's the ultimate treat for Benedict Cumberbat...   
8   You would be forgiven for thinking these colou...   
9   One-eyed police murderer Dale Cregan (above) i...   
10  The NFL has its first full-time female game of...   
11  One-eyed police killer Dale Cregan has been se...   
12  For any players who have featured for 90 minut...   
13  Gary Neville has hailed Manchester United as a...   
14  Eden Hazard needs to become more selfish if he...   
15  Famished and queuing despondently as far as th...   
16  The country’s biggest food bank organisation y...   
17  (CNN)An Egyptian court sentenced the leader of...   
18  We're all guilty of it but there are some who ...   
19  Anthony Joshua believes his 'hunger and determ...   

                                   _source.highlights  
0   Motiongate Dubai will open in October, 2016 wi...  
1   Young adult novels regularly feature teen hero...  
2   Hunger Games star Josh Hutcherson, 22, surpris...  
3   The final film featuring the late Paul Walker,...  
4   Films can be bought at Skystore.com or through...  
5   The film is expected to gross $115 million or ...  
6   Jose Mourinho said he is proud to work with un...  
7   A 6ft chocolate sculpture of Benedict Cumberba...  
8   Wellington held its inaugural CubaDupa festiva...  
9   31-year-old was transferred to solitary confin...  
10  Thomas will be a line judge for the 2015 seaso